# Twitter Scraping

This Python Notebook involves code that target scraping timeline tweets from a list of users.
A twitter developer account is required to access the twitter API.

In [2]:
consumer_key = "r31n3QB8yraZK0cDXI1IGR4TB"
consumer_secret = "yqp4Zl6H6yxz7hq0oherdDaVQDS4wKukSZXO8zqb5zl4xwAO9c"
access_token = "1377235714891415555-VijVHZt5q2cJSTMkGM9Hn8dUahwntR"
access_token_secret = "BKcvTcfi8vhvTJ8czfK4UdRW4TaqWi5lj7hfSU6xXfgEY"

## Step 1: Initialize Tweepy Instance
For small scraping jobs or if the user has access to a premium Twitter developer account,
the extra parameters aside from the auth keys are unnecessary.

In [4]:
import tweepy
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, retry_count = 1, retry_errors = [408], wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

## Step 2: Reading Tweets from a Pre-Curated List of Twitter Users

In [16]:
import pandas as pd

user_ids = pd.read_csv("user_ids.csv", header = None)
user_ids.head()

,0
0,728973854287101952
1,1315729337723879425
2,824797212425191425
3,1240775744088768512
4,785520949105950724


# Step 3: Scraping Tweets

Note: 

This notebook uses an incremental appending method to append tweets to a csv file.

For small scraping jobs (~ less than 100k tweets), it would be completely unnecessary, and all tweets can be directly added to a list and then be exported.

However, in cases of large scraping jobs, such as in the example user_id list where the final dataset is ~12.9gb, the notebook will run into issues if the
size of the data exceeds the computers memory. As such, the incremental appending approach will help to ensure the least risk of data loss.

Additionally, in this project, a half dataset was made which included fewer data.


In [6]:
import os.path

all_tweets = []

for i, user_ids in enumerate(user_ids[0]):
    try: 
        # Print Progress
        print(f"Getting tweets from {user_id}, user {i + 1}/{len(user_ids[0])}")

        # Initialize current user's tweets list
        user_tweets = []

        # Get first chunk of tweets from user
        new_tweets = api.user_timeline(user_id = user_id, count = 200, trim_user = True, exclude_replies = True, include_rts = False)
        
        # Add tweets to the user's tweets list
        user_tweets.extend(new_tweets)

        # Loop until there are no tweets left
        while len((new_tweets)) > 0:

            #update the id of the oldest tweet less one
            oldest = user_tweets[-1].id - 1
            
            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(user_id = user_id, count = 200, trim_user = True, exclude_replies = True, include_rts = False, max_id = oldest)
            
            #save most recent tweets
            user_tweets.extend(new_tweets)

        # Print Progress
        print(f"Added total: {len(user_ids[0])} tweets from {user_id}")

        # Add Tweets to All Tweets list
        all_tweets.extend(user_tweets)

        # Incremental Append: If tweets length greater than 50000, save to csv
        if len(all_tweets) > 50000:

            # Map Tweepy Tweet Object to json format for easy conversion to DataFrame
            df = pd.DataFrame(map(lambda x: x._json, all_tweets))

            # Half Dataset containing fewer attributes
            df2 = df[['created_at', 'text', 'user', 'retweet_count', 'favorite_count']]

            if not os.path.exists("full.csv"):
                df.to_csv("full.csv")
                df2.to_csv("half.csv")
            else:
                df.to_csv("full.csv", mode = 'a', header = False)
                df2.to_csv("half.csv", mode = 'a', header = False)

            all_tweets = []
            print("Data concatenated to CSVs")

    # Error Handling
    except tweepy.TweepError as e: 
        print(f"Tweepy Error: {e}, with user {user_id}")
    except Exception as e:
        print(f"Unknown Error: {e}, from user {user_id}")

# Add excess tweets to csv
df = pd.DataFrame(map(lambda x: x._json, all_tweets))
df2 = df[['created_at', 'text', 'user', 'retweet_count', 'favorite_count']]

df.to_csv("full.csv", mode = 'a', header = False)
df2.to_csv("half.csv", mode = 'a', header = False)

s from 41073372, user 3774/4001
Added total: 2941 tweets from 41073372
Data concatenated to CSVs
Getting tweets from 2601630428, user 3775/4001
Added total: 2893 tweets from 2601630428
Getting tweets from 14301922, user 3776/4001
Added total: 599 tweets from 14301922
Getting tweets from 106871524, user 3777/4001
Added total: 1204 tweets from 106871524
Getting tweets from 3010738918, user 3778/4001
Tweepy Error: Not authorized., with user 3010738918
Getting tweets from 154221292, user 3779/4001
Added total: 1118 tweets from 154221292
Getting tweets from 14433007, user 3780/4001
Added total: 2319 tweets from 14433007
Getting tweets from 72235760, user 3781/4001
Added total: 2128 tweets from 72235760
Getting tweets from 735983, user 3782/4001
Added total: 505 tweets from 735983
Getting tweets from 21969650, user 3783/4001
Added total: 1196 tweets from 21969650
Getting tweets from 403061491, user 3784/4001
Tweepy Error: Not authorized., with user 403061491
Getting tweets from 4273224434, u